# Q&A over LangChain Docs

Let's evaluate your architecture on a Q&A dataset for the LangChain python docs.


## Pre-requisites

We will install quite a few prerequisites for this example since we are comparing various techinques and models.

In [ ]:
# %pip install -U langchain_benchmarks
# %pip install -U langchain langsmith langchainhub chromadb openai huggingface pandas langchain_experimental

In [1]:
%load_ext autoreload
%autoreload 2

For this code to work, please configure LangSmith environment variables with your credentials.

In [2]:
import os

os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "sk-..."  # Your API key

# Silence warnings from HuggingFace
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Review Q&A "environments"

The registry provides configurations to test out common architectures on curated datasets.

In [3]:
from langchain_benchmarks import clone_public_dataset, registry

In [4]:
registry

Name,Dataset ID,Description
Tool Usage - Typewriter (1 func),placeholder,"Environment with a single function that accepts a single letter as input, and ""prints"" it on a piece of paper. The objective of this task is to evaluate the ability to use the provided tools to repeat a given input string. For example, if the string is 'abc', the tools 'a', 'b', and 'c' must be invoked in that order. The dataset includes examples of varying difficulty. The difficulty is measured by the length of the string."
Tool Usage - Typewriter,placeholder,"Environment with 26 functions each representing a letter of the alphabet. In this variation of the typewriter task, there are 26 parameterless functions, where each function represents a letter of the alphabet (instead of a single function that takes a letter as an argument). The object is to evaluate the ability of use the functions to repeat the given string. For example, if the string is 'abc', the tools 'a', 'b', and 'c' must be invoked in that order. The dataset includes examples of varying difficulty. The difficulty is measured by the length of the string."
Tool Usage - Relational Data,e95d45da-aaa3-44b3-ba2b-7c15ff6e46f5,"Environment with fake data about users and their locations and favorite foods. The environment provides a set of tools that can be used to query the data. The objective of this task is to evaluate the ability to use the provided tools to answer questions about relational data. The dataset contains 21 examples of varying difficulty. The difficulty is measured by the number of tools that need to be used to answer the question. Each example is composed of a question, a reference answer, and information about the sequence in which tools should be used to answer the question. Success is measured by the ability to answer the question correctly, and efficiently."
Multiverse Math,placeholder,"An environment that contains a few basic math operations, but with altered results. For example, multiplication of 5*3 will be re-interpreted as 5*3*1.1. The basic operations retain some basic properties, such as commutativity, associativity, and distributivity; however, the results are different than expected. The objective of this task is to evaluate the ability to use the provided tools to solve simple math questions and ignore any innate knowledge about math."
Email Extraction,https://smith.langchain.com/public/36bdfe7d-3cd1-4b36-b957-d12d95810a2b/d,"A dataset of 42 real emails deduped from a spam folder, with semantic HTML tags removed, as well as a script for initial extraction and formatting of other emails from an arbitrary .mbox file like the one exported by Gmail. Some additional cleanup of the data was done by hand after the initial pass. See https://github.com/jacoblee93/oss-model-extraction-evals."
LangChain Docs Q&A,452ccafc-18e1-4314-885b-edd735f17b9d,Questions and answers based on a snapshot of the LangChain python docs. The environment provides the documents and the retriever information. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).


In [5]:
langchain_docs = registry["LangChain Docs Q&A"]
langchain_docs

Name,LangChain Docs Q&A
Type,RetrievalTask
Dataset ID,452ccafc-18e1-4314-885b-edd735f17b9d
Description,Questions and answers based on a snapshot of the LangChain python docs. The environment provides the documents and the retriever information. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).
Retriever Factories,"basic, parent-doc, hyde"
Architecture Factories,conversational-retrieval-qa


In [6]:
clone_public_dataset(langchain_docs.dataset_id, dataset_name=langchain_docs.name)

Dataset LangChain Docs Q&A already exists. Skipping.
You can access the dataset at http://localhost/o/00000000-0000-0000-0000-000000000000/datasets/1e4bf58b-1a61-44fb-bb84-4c5c0e2b4b5b.


In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-base")

retriever_factory = langchain_docs.retriever_factories["basic"]
# Indexes the documents with the specified embeddings
# Note that this does not apply any chunking to the docs,
# which means the documents can be of arbitrary length
retriever = retriever_factory(embeddings)

0it [00:00, ?it/s]

In [8]:
# Factory for creating a conversational retrieval QA chain

chain_factory = langchain_docs.architecture_factories["conversational-retrieval-qa"]

In [9]:
from langchain.chat_models import ChatAnthropic

# Example
llm = ChatAnthropic(model="claude-2", temperature=1)

chain_factory(retriever, llm=llm).invoke({"question": "what's lcel?"})

' Here is a summary of LCEL (LangChain Expression Language) based on the provided search results:\n\n- LCEL is a declarative way to easily compose chains together in Langchain. It supports streaming, async, optimized parallel execution, retries, fallbacks, access to intermediate results, input/output schemas, and integration with LangSmith tracing and LangServe deployment. [0]\n\n- LCEL enables putting prototypes into production with no code changes, from simple prompt + LLM chains to complex 100+ step chains. [0]  \n\n- Key benefits include minimized time-to-first-token, ability to call chains sync or async with great performance, automatic parallelization, configuring reliability features like retries, accessing intermediate outputs for debugging/streaming, and easy LangSmith tracing + LangServe deployment. [0]'

### Evaluate

Let's evaluate a retriever now.

In [12]:
from functools import partial

from langsmith.client import Client

from langchain_benchmarks.rag import get_eval_config

In [16]:
client = Client()
RAG_EVALUATION = get_eval_config()

test_run = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=partial(chain_factory, retriever, llm=llm),
    evaluation=RAG_EVALUATION,
    verbose=True,
)

View the evaluation results for project 'test-crushing-pain-100' at:
http://localhost/o/00000000-0000-0000-0000-000000000000/projects/p/c3dbdc97-39b9-44ab-9367-2644ffd7fc30?eval=true

View all tests for Dataset LangChain Docs Q&A at:
http://localhost/o/00000000-0000-0000-0000-000000000000/datasets/1e4bf58b-1a61-44fb-bb84-4c5c0e2b4b5b
[>                                                 ] 0/86


KeyboardInterrupt



In [ ]:
test_run.get_aggregate_feedback()